# <span style="font-width:bold; font-size: 3rem; color:#1EB182;">**Hopsworks Feature Store** </span> <span style="font-width:bold; font-size: 3rem; color:#333;">- Part 04: Model training & UI Exploration</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/electricity/4_model_training_and_registration.ipynb)


## 🗒️ This notebook is divided into 3 main sections:
1. **Loading the training data**.
2. **Train the model**.
3. **Register model to Hopsworks model registry**.

![tutorial-flow](https://github.com/logicalclocks/hopsworks-tutorials/blob/master/images/03_model.png?raw=1)

### <span style="color:#ff5f27;">📝 Importing Libraries</span>

In [ ]:
!pip install -U hopsworks --quiet

In [ ]:
!pip install -U xgboost --quiet

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
import xgboost as xgb

from sklearn.metrics import r2_score

import warnings
warnings.filterwarnings('ignore')

---

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store()

---

## <span style="color:#ff5f27;">🪝 Feature View and Training Dataset Retrieval</span>

In [ ]:
feature_view = fs.get_feature_view(
    name = 'citibike_fv',
    version = 1
)

In [ ]:
import pandas as pd

X_train, X_test, y_train, y_test = feature_view.get_train_test_split(1)

In [ ]:
X_train.iloc[:, 1:-1] = X_train.iloc[:, 1:-1].astype(float)
X_test.iloc[:, 1:-1] = X_test.iloc[:, 1:-1].astype(float)

In [ ]:
X_train.shape

In [ ]:
y_train.shape

In [ ]:
X_train = X_train.set_index(["date", "station_id"])
X_test = X_test.set_index(["date", "station_id"])

In [ ]:
X_train.head(3)

---

## <span style="color:#ff5f27;">🧬 Modeling</span>

In [ ]:
regressor = xgb.XGBRegressor()
 
# Fitting the model
regressor.fit(X_train, y_train)
 
# Predict the model
y_pred = regressor.predict(X_test)
 
r2_xgb = r2_score(y_pred, y_test.values)
print("R2 score for XGBoost model:", r2_xgb)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


df_ = pd.DataFrame({
    "y_true": y_test,
    "y_pred": y_pred
})

residplot = sns.residplot(data=df_, x="y_true", y="y_pred", color='#613F75')
plt.title('Model Residuals')
plt.xlabel('Obsevation #')
plt.ylabel('Error')

plt.show()

In [ ]:
import os

if os.path.isdir("assets") == False:
    os.mkdir("assets")


fig = residplot.get_figure()
fig.savefig("assets/residplot.png") 
fig.show()

---

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X_train)
output_schema = Schema(y_train)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

## <span style='color:#ff5f27'>🗄 Model Registry</span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

In [ ]:
import joblib
import os
import shutil


# The 'nyc_taxi_fares_model' directory will be saved to the model registry
model_dir="citibike_xgb_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)

joblib.dump(regressor, model_dir + '/citibike_xgb_model.pkl')

shutil.copyfile("assets/residplot.png", model_dir + "/residplot.png")

In [ ]:
mr = project.get_model_registry()

citibike_model = mr.python.create_model(
    name="citibike_xgb_model", 
    metrics={"r2_score": r2_xgb},
    model_schema=model_schema,
    input_example=X_train.sample(), 
    description="Citibike users per station Predictor")

citibike_model.save(model_dir)